# `Pre-processing`

## 1 Setup

### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np

### 1.2 Paths

In [3]:
DEPRESSION_PATH = '/workspaces/depressao-ibge-pns/data/depression.csv'


### 1.3 Dataframes

In [6]:
depression = pd.read_csv(DEPRESSION_PATH, header=[0, 1])

In [7]:
depression

,V0001,V0026,V0031,C006,C00703,C008,C009,C01001,C013,C017,...,V033,V034,V03501,V03502,V03503,V036,V037,V038,V039,Y008
,Unidade da Federação,Tipo de situação censitária,Tipo de área,Sexo,Ano de nascimento,Idade do morador na data de referência,Cor ou raça,Cônjuge ou companheiro(a) mora em nesse domicílio.,Cônjuge ou companheiro(a) mora em outro domicílio.,___já viveu com cônjuge ou companheiro (a) antes?,...,Onde isso ocorreu?,"Nos últimos doze meses, você deixou de realizar quaisquer de suas atividades habituais (trabalhar, realizar afazeres domésticos, ir à escola etc.) por causa desse ato","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Hematomas, cortes, fraturas, queimaduras ou outras lesões físicas ou ferimentos?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Medo, tristeza, desânimo, dificuldade para dormir, ansiedade, depressão ou outras consequências psicológicas?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Doença sexualmente transmissível ou gravidez indesejada?","Por causa desta (s) consequência (s), você procurou algum atendimento de saúde","Por causa desta (s) consequência (s), você recebeu algum atendimento de saúde",Onde foi realizado este atendimento de saúde,"Por causa desta (s) consequência (s), você precisou ser internado por 24 horas ou mais",Qual é a sua orientação sexual?
0,11,1,1,2.0,1963.0,55.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,11,1,1,1.0,1950.0,69.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1,1,1.0,1987.0,31.0,2.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,1,1,1.0,9999.0,9.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1,1,2.0,9999.0,6.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293721,53,2,1,2.0,1970.0,49.0,2.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293722,53,2,1,1.0,1935.0,84.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293723,53,2,1,1.0,1997.0,21.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5 Implicit `NaNs`
> Useless classes such as '09 - Ignored' become NaNs

In [ ]:
depression.C00703

Descricao,Ano de nascimento
0,1963
1,1950
2,1987
3,9999
4,9999
...,...
293721,1970
293722,1935
293723,1997
293724,2006


In [ ]:
depression[depression.C00703 == pd.NA]

: 

## 6 Class `mapping`

## 7 D`types`

## `TODO`: Implicit nulls
> Apparently, depression.dropna(how='all') won't drop everything we need, <br>
 because of useless classes such as "01 - can't remember" or "09 - don't know".

### 1.1 Filter the dictionary.Descricao if value in depression.column

In [ ]:
depression.columns

#  TODO:
#  THERE ARE 3 COLUMNS ENDING WITH .1
    # 'Peso - Final (em kg)(3 inteiros e 1 casa decimal).1',
    # 'Que idade ___tinha quando começou a viver com seu(sua) primeiro(a)/único(a) marido (mulher) ou companheiro(a)?.1',
    # 'Quem fez isso com você.1'

MultiIndex([(  'V0001', ...),
            (  'V0026', ...),
            (  'V0031', ...),
            (   'C006', ...),
            ( 'C00703', ...),
            (   'C008', ...),
            (   'C009', ...),
            ( 'C01001', ...),
            (   'C013', ...),
            (   'C017', ...),
            (   'C018', ...),
            ( 'C01801', ...),
            (   'C011', ...),
            ( 'D00901', ...),
            ( 'I00102', ...),
            (   'I005', ...),
            ( 'I00401', ...),
            ( 'I00403', ...),
            ( 'I00404', ...),
            ('M005010', ...),
            (   'M009', ...),
            ( 'M01401', ...),
            ( 'M01501', ...),
            ( 'M01601', ...),
            ( 'M01901', ...),
            (   'N016', ...),
            ( 'P00102', ...),
            ( 'P00103', ...),
            ( 'P00104', ...),
            (   'P027', ...),
            ( 'P02801', ...),
            (   'P034', ...),
            (   'P035', ...),
          

In [ ]:
# Filter the dictionary to only include columns that are in the depression dataset
depression_dictionary = dictionary[dictionary.Descricao.isin(depression.columns)]

NameError: name 'dictionary' is not defined

In [ ]:
depression_dictionary.to_excel(
    '../data/depression_dictionary.xlsx',
    index=False,
)

In [ ]:
depression_dictionary

## `TODO` Drop columns without class descriptions
Codigo `V03501` is not correctly described in the dictionary-pns-2019-cleaned.csv file.

In [ ]:
# V03501
PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo == 'V03501']

In [ ]:
# Original shape
print(PNS_DICTIONARY_PATH.shape)

# Drop Codigo 'V03501'
PNS_DICTIONARY_PATH = PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo != 'V03501']

# Shape after dropping
print(PNS_DICTIONARY_PATH.shape)